In [1]:
# these four imports will provide most of the functionality required in 
# to start working with basler cameras
# pypylon 
import pypylon.pylon as py
# plotting for graphs and display of image
import matplotlib.pyplot as plt
# linear algebra and basic math on image matrices
import numpy as np

import pypylon.pylon as py
import numpy as np
import matplotlib.pyplot as plt

# handle exception trace for debugging 
# background loop
import traceback

import time
import random

In [2]:
# get instance of the pylon TransportLayerFactory
tlf = py.TlFactory.GetInstance()
devices = tlf.EnumerateDevices()
cam = py.InstantCamera(tlf.CreateDevice(devices[0]))

cam.Open()

# to get consistant results it is always good to start from "power-on" state
cam.UserSetSelector.Value = "Default"
cam.UserSetLoad.Execute()

cam.ExposureTime.Value = cam.ExposureTime.Min

/tmp/ipykernel_5648/2578635988.py:9: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.UserSetSelector.Value = Default
  cam.UserSetSelector = "Default"
/tmp/ipykernel_5648/2578635988.py:12: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ExposureTime.Value = 16.0
  cam.ExposureTime = cam.ExposureTime.Min


In [3]:
class ImageHandler(py.ImageEventHandler):
    def __init__(self):
        super().__init__()
        self.img_sum = np.zeros((cam.Height.Value, cam.Width.Value), dtype=np.uint16)
        
    def OnImageGrabbed(self, camera, grabResult):
        """ we get called on every image
            !! this code is run in a pylon thread context
            always wrap your code in the try .. except to capture
            errors inside the grabbing as this can't be properly reported from 
            the background thread to the foreground python code
        """
        try:
            if grabResult.GrabSucceeded():
                # check image contents
                img = grabResult.Array
                self.img_sum += img
            else:
                raise RuntimeError("Grab Failed")
        except Exception as e:
            traceback.print_exc()

In [4]:
def BackGroundLoopSample():

    # instantiate callback handler
    handler = ImageHandler()
    # register with the pylon loop
    cam.RegisterImageEventHandler( handler , py.RegistrationMode_ReplaceAll, py.Cleanup_None)

    # fetch some images with background loop
    cam.StartGrabbingMax( 100, py.GrabStrategy_LatestImages, py.GrabLoop_ProvidedByInstantCamera)
    while cam.IsGrabbing():
        # random exposuretime changes every 100ms
        cam.ExposureTime = random.uniform(cam.ExposureTime.Min, 1000)
        time.sleep(0.1)

    cam.StopGrabbing()
    cam.DeregisterImageEventHandler(handler)
    
    return handler.img_sum

In [5]:
%%timeit -o
BackGroundLoopSample()

/tmp/ipykernel_5648/78701421.py:12: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ExposureTime.Value = 600.3521718696645
  cam.ExposureTime = random.uniform(cam.ExposureTime.Min, 1000)
/tmp/ipykernel_5648/78701421.py:12: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ExposureTime.Value = 315.8135461243162
  cam.ExposureTime = random.uniform(cam.ExposureTime.Min, 1000)
/tmp/ipykernel_5648/78701421.py:12: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ExposureTime.Value = 893.8378916137008
  cam.ExposureTime = random.uniform(cam.ExposureTime.Min, 1000)
/tmp/ipykernel_5648/78701421.py:12: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ExposureTime.Value = 282.41693449978806
  cam.ExposureTime = random.uniform(cam.ExposureTime.Min, 1000)
/tmp/ipykernel_5648/78701421.py:12: DeprecationWarning: Setting a f

23.9 s ± 19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 23.9 s ± 19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [6]:
background_average = _.average / 100

In [7]:
f"average time to capture one frame is: {background_average*1e6:.2f} µs"

'average time to capture one frame is: 239332.18 µs'

In [8]:
cam.StopGrabbing()
cam.Close()